# Training Sólo Escúchame: Spanish Emotional Accompaniment Chatbot 💬🤖

This notebook presents the final iteration of the training process, which was part of the experimental phase reported in the article [Sólo Escúchame: Spanish Emotional Accompaniment Chatbot](https://arxiv.org/).

This model uses [Llama2-7b-chat](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) as the base model, fine-tuning it for the task of emotional accompaniment. The training utilizes the [HEAR](https://huggingface.co/datasets/BrunoGR/HEAR-Hispanic_Emotional_Accompaniment_Responses) (Hispanic Emotional Accompaniment Responses) dataset, proposed in the same article as this model. The objective is to tailor the model's responses to provide emotional support in various situations.

HEAR is based on another dataset, [HRECPW](https://huggingface.co/datasets/BrunoGR/HRECPW-Hispanic_Responses_for_Emotional_Classification_based_on_Plutchik_Wheel), which compiles elements from datasets focused on emotion tagging and classification. It unites these elements into a dataset with 11 emotion categories: Affection, Joy, Admiration, Anger, Sadness, Optimism, Hatred, Surprise, Fear, Calm, and Disgust.

The elements comprising HEAR were sampled randomly from HRECPW, and the responses for these elements were generated using OpenAI's GPT-3.5-Turbo API.

## Let´s start to code
1. It is needed to install some dependencies

In [ ]:
!pip install -r ../requirements.txt

2. import the installed dependencies

In [ ]:
from transformers import (
    TrainingArguments, 
    EarlyStoppingCallback, 
    LlamaTokenizer, 
    AutoModelForCausalLM, 
)
from peft import ( 
    LoraConfig, 
    PeftModel,
    get_peft_model
)
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from huggingface_hub import login  
import pandas as pd
import wandb, datasets, os, load_dotenv


3. load the environmental variables as access tokens for wandb and huggingface_hub to track the training progress and plot the metrics, and access to models and datasets. 

In [ ]:
load_dotenv.load_dotenv()
access_token = os.getenv('acesstoken')
login(token=access_token)

## Configure the Model for training
1. Load the model: In this iteration a A100 of 80GB of VRAM was used for ~18hrs so we dispensed to quantized the 7 billion parameter model. The goal is to get all model parameters with the most precision that 32 bit can give. 
2. tie_weights(): To maximize the efficiency of memory when the model is loaded, tie the weights between the input embedding layer and the output embedding layer with [tie_weights()](https://huggingface.co/docs/transformers/main_classes/model#transformers.PreTrainedModel.tie_weights). 
3. Load the Tokenizer.


In [ ]:
parameters="7b-chat"
BASE_MODEL = f"meta-llama/Llama-2-{parameters}-hf"
# if there is a pretrained model, load it the model is Models_of_Llama/Llama_base
myModel= "BrunoGR/EmotionalBot_LLaMA2"
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    #torch_dtype=torch.float16,
    #quantization_config=bnb_config,
    #load_in_8bit=True,
    device_map="auto"
)
model.tie_weights()
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer =   LlamaTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token

4.  Load the dataset

In [ ]:
data= datasets.load_dataset("BrunoGR/HEAR-Hispanic_Emotional_Accompaniment_Responses")

5. Set the LoRA Parameters, to acquire a bigger adapter to modify the model weights during the training process, the 'r' Value must be increased, and to get a bigger scale of modification to model weights increase the lora_alpha is necessary. Because Llama2 is not completely multilingual, significant changes must be made to acquire better results.  

In [6]:
LoRA_TARGET_MODULES = [ 
    "q_proj", 
    "v_proj",
]

LoRA_DROPOUT= 0.1
config = LoraConfig(
    r=64,
    lora_alpha=128,
    target_modules=LoRA_TARGET_MODULES,
    lora_dropout=LoRA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)

(optional) If you want to see how many parameters are going to be trained disable the '''. 

In [ ]:
'''
pftmdl = get_peft_model(model, config)
pftmdl.print_trainable_parameters()
'''

### 6. The training parameters:

The model training is configured with a total batch size of $15$ and a micro batch size of $5$ to optimize GPU memory and training stability, accumulating gradients over $3$ steps to simulate a larger batch size. A warm-up of $300$ steps is used to stabilize the learning rate, starting at $5e-5$, with AdamW hyperparameters recommended in the LLaMA paper, including adam_beta1 of $0.9$, adam_beta2 of $0.95$, and adam_epsilon of $2e-8$. Weight decay of $0.1$ is applied to prevent overfitting, and metrics are logged every $10$ steps. The model is evaluated and saved every $450$ steps, with a limit of $6$ saved checkpoints. Gradient normalization is set to a maximum of $1.0$, and a cosine learning rate scheduler adjusts the rate dynamically during training. The configuration also includes loading the best model at the end and reporting results to Weights & Biases for detailed tracking.

In [7]:
BATCH_SIZE = 15 # tamaño del batch, es decir, cuantos textos se procesan a la vez
MICRO_BATCH_SIZE = 5# tamaño del micro batch, es decir, cuantos textos se procesan a la vez en la GPU
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE # pasos de acumulación de gradientes
training_arguments = TrainingArguments( # se configuran los argumentos de entrenamiento
    per_device_train_batch_size=MICRO_BATCH_SIZE, # tamaño del micro batch
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS, # pasos de acumulación de gradientes
    warmup_steps=300, # pasos de calentamiento del entrenamiento
    num_train_epochs = 3, # epocas de entrenamiento que son 300
    learning_rate=5e-5, # tasa de aprendizaje
    adam_beta1=0.9, # betas de adam, se usa el mismo del paper de llama
    adam_beta2=0.95, # se usa el mismo del paper de llama
    adam_epsilon=2e-8, # se usa el mismo del paper de llama
    weight_decay=0.1,
    #fp16=True, # se usa la precisión de 16 bits
    logging_steps=10, # pasos de logging
    optim="adamw_torch", # optimizador adamw, se usa el de torch
    evaluation_strategy="steps", # estrategia de evaluación
    save_strategy="steps", # estrategia de guardado
    eval_steps=450, # cada 50 pasos se evalúa el modelo
    save_steps=450, # cada 50 pasos se guarda el modelo
    output_dir="checkpoint-41_9k-lr5em5-bs15n5_r64LA128dt01_2811", # directorio de salida
    save_total_limit=6, # límite de guardado
    load_best_model_at_end=True, #se guarda  el mejor modelo al final
    #metric_for_best_model= "accuracy",
    report_to="wandb", # se reporta a wandb
    seed=1,
    lr_scheduler_type = "cosine",# tal y como dice en el paper de llama
    max_grad_norm = 1.0, # tal y como dice en el paper de llama
)

7. initialize wandb tracking

In [ ]:
#os.environ['WANDB_API_KEY'] = 'Your token here if you don't have any .env file'
wandb.init(project="prueba", name="emo-41_9k-lr5em5-bs15n5_r64LA128dt01")

### 8. DataCollatorForCompletionOnlyLM configuration
- The data collator is configured to handle input sequences for language model completion tasks. The tokenizer.pad_token_id is set to tokenizer.eos_token_id + 1 to ensure proper padding during batching, where the padding token is distinct from the end-of-sequence token. 
- The response_template_with_context defines a template for generating responses with context, while instruction_template specifies the prefix for instructions. The response_template_ids are created by encoding the response_template_with_context without special tokens, resulting in token IDs that represent the response template. This is crucial for aligning with how the dataset texts are encoded. 
- DataCollatorForCompletionOnlyLM is instantiated with response_template_ids, instruction_template, and the tokenizer. This collator handles the preparation of input batches, ensuring that the response templates and instructions are correctly integrated into the data fed to the model.

In [9]:
tokenizer.pad_token_id = tokenizer.eos_token_id + 1
response_template_with_context = "\n### response:"
instruction_template="instruction:"
response_template_ids = tokenizer.encode(response_template_with_context, add_special_tokens=False)[2:]  # Now we have it like in the dataset texts: `[2277, 29937, 4007, 22137, 29901]`
print(response_template_ids)
collator = DataCollatorForCompletionOnlyLM(response_template_ids,instruction_template, tokenizer=tokenizer)

[2277, 29937, 2933, 29901]


9. The `SFTTrainer` is configured for training a model with specific parameters and callbacks. Here's a breakdown of the configuration:

- **`model`**: The model to be trained.
- **`data_collator`**: The `collator` is used to handle the preparation of batches, ensuring proper formatting and integration of templates in the training data.
- **`train_dataset`**: The dataset used for training, specified by `data['train']`.
- **`eval_dataset`**: The dataset used for evaluation, specified by `data['validation']`.
- **`peft_config`**: Configuration for Parameter-Efficient Fine-Tuning (PEFT), provided by `config`.
- **`dataset_text_field`**: Specifies the field in the dataset that contains the text prompts, set to `"Prompt_en"`.
- **`max_seq_length`**: Defines the maximum sequence length for input sequences, set to 824 tokens.
- **`tokenizer`**: The tokenizer used to process the text data.
- **`args`**: Training arguments configured for the training process, including batch size, learning rate, and other hyperparameters.
- **`callbacks`**: Includes `EarlyStoppingCallback` with a patience of 3, which stops training if no improvement is observed for 3 evaluation steps.

This setup ensures that the model is trained with well-defined data handling and evaluation procedures, optimized for efficiency and early stopping to prevent overfitting.

In [ ]:
trainer = SFTTrainer(
    model=model,
    data_collator= collator,
    train_dataset=data['train'],
    eval_dataset =data['validation'],
    peft_config=config,
    dataset_text_field="Prompt_en",
    max_seq_length=824,
    tokenizer=tokenizer,
    args=training_arguments,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

### 10. Let it train 
Perfect time for tea and nap, or play some videogame's campaign, you have 18 hours till complete so...
here some recommendations:
1. **The Last of Us Part II** - A deep and emotional story in a post-apocalyptic world.
2. **God of War (2018)** - A mythological adventure with Kratos and his son Atreus.
3. **Resident Evil 4/Remake** - A classic survival horror game with an intense campaign.
4. **Uncharted 4: A Thief's End** - An exciting action-adventure with Nathan Drake.
5. **Dead Space/Remake** - The best survival horror ever.

In [ ]:
trainer.train(resume_from_checkpoint=True) # se entrena el modelo

Step,Training Loss,Validation Loss
450,0.771700,0.906528
900,0.771600,0.886262
1350,0.785600,0.867941
1800,0.746300,0.860214


In [ ]:
trainer.train(resume_from_checkpoint=True)

Step,Training Loss,Validation Loss
5400,0.668400,0.827485


In [ ]:
trainer.train(resume_from_checkpoint=True)

Step,Training Loss,Validation Loss
5850,0.639500,0.826833
6300,0.673500,0.827229


In [ ]:
trainer.train(resume_from_checkpoint=True)

Step,Training Loss,Validation Loss
6750,0.641600,0.827501
7200,0.651000,0.826495


In [11]:
trainer.train(resume_from_checkpoint=True)

Step,Training Loss,Validation Loss
7650,0.651400,0.826496
8100,0.653500,0.826495


TrainOutput(global_step=8382, training_loss=0.09142078984412337, metrics={'train_runtime': 9461.6276, 'train_samples_per_second': 13.288, 'train_steps_per_second': 0.886, 'total_flos': 1.4362559087474688e+18, 'train_loss': 0.09142078984412337, 'epoch': 3.0})

My lord, I did not expect that it would take so long. At least we made it. Let´s test it. 

11. save the adapter and push it to huggingface hub

In [ ]:
trainer.save_model("Adptr-41_9k-lr5em5-bs15n5_r64LA128dt01_2811")
trainer.push_to_hub("JUST_HEAR_ME-PEFT_Adapter")

# Fast Model Test
1. Let's first load the adapter and merge it with the original model

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    #torch_dtype=torch.float16,
    #quantization_config=bnb_config,
    #load_in_8bit=True,
    device_map="auto"
)
model.tie_weights()
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer =   LlamaTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token

#load the adapter and merge it.
model= PeftModel.from_pretrained(model,"BrunoGR/JUST_HEAR_ME-PEFT_Adapter")
final=model.merge_and_unload()

In [16]:
instructions = [
    "Eres un asistente emocional, responde de forma respetuosa y adecuada a la situación emocional del usuario. Si el usuario parece estar triste o molesto, el asistente debe responder de manera empática y ofrecer palabras de aliento. Si el usuario parece estar feliz o emocionado, el asistente debe compartir esa alegría y responder de manera entusiasta. En todos los casos, el asistente debe mantener un tono respetuoso y profesional.",
    "Eres un asistente emocional, responde de forma respetuosa y adecuada a la situación emocional del usuario. Si el usuario parece estar triste o molesto, el asistente debe responder de manera empática y ofrecer palabras de aliento. Si el usuario parece estar feliz o emocionado, el asistente debe compartir esa alegría y responder de manera entusiasta. En todos los casos, el asistente debe mantener un tono respetuoso y profesional.",
    "You are an emotional assistant, respond empathetically in Spanish to each of the messages. If the user seems sad or upset, you should offer words of encouragement. If the user seems happy or excited, the assistant should share that joy and respond enthusiastically. In all cases, the assistant should maintain a respectful tone, if possible, encouraging you to talk more about it. Don't say hello, unless necessary. Use the username and pronoun to respond in a personalized way.",

]
inputs= [
    "En serio que como me caga que haga las cosas y no me las cuente, me las oculte."
    ,'''
        {
        "usuario":"Jannette",
        "Pronombre":"Ella",
        "Mensaje":"En serio que como me caga que haga las cosas y no me las cuente, me las oculte."
        }
    ''',
    '{"usuario":"Jannette","Pronombre":"Ella","Mensaje":"Hola, me siento molesta el dio de hoy."}\n{"usuario":"Asistente Emocional","Pronombre":"Él","Mensaje":"Hola Jannette, que mal que el dia de hoy no te sientas bien, ¿Podrias contarme que es lo que te molesta?"}\n{"usuario":"Jannette","Pronombre":"Ella","Mensaje":"En serio que como me caga que mi novio haga las cosas y no me las cuente, me las oculte."}\n{"usuario":"Asistente Emocional","Pronombre":"Él","Mensaje":"Entiendo que te sientas frustrada cuando alguien oculta cosas importantes. ¿Puedes compartir más sobre lo que está pasando?"}\n{"usuario":"Jannette","Pronombre":"Ella","Mensaje":"Es que siento que no confía en mí para contármelo, y eso me hace sentir excluida."}'

]
def response (query:str,maxtoken:int):
    input_ids = tokenizer(query, return_tensors="pt").input_ids.to('cuda')
    ntok=len(tokenizer.tokenize(query))
    generation_output = final.generate(
        input_ids=input_ids, max_new_tokens=maxtoken
    )
    out = tokenizer.decode(generation_output[0] , skip_special_tokens=True)
    return f"Numero de tokens de entrada:{ntok}\n\nSalida:\n{out}"

In [18]:
input=inputs[2]
prompt = f'''Below is an instruction that describes a task, paired with an input that provides further context. 

### Instruction:
{instructions[2]}\nskip emojis.

### Input:
{input}

### Response:'''
a=response(prompt,100)
print(a)

Numero de tokens de entrada:409

Salida:
Below is an instruction that describes a task, paired with an input that provides further context. 

### Instruction:
You are an emotional assistant, respond empathetically in Spanish to each of the messages. If the user seems sad or upset, you should offer words of encouragement. If the user seems happy or excited, the assistant should share that joy and respond enthusiastically. In all cases, the assistant should maintain a respectful tone, if possible, encouraging you to talk more about it. Don't say hello, unless necessary. Use the username and pronoun to respond in a personalized way.
skip emojis.

### Input:
{"usuario":"Jannette","Pronombre":"Ella","Mensaje":"Hola, me siento molesta el dio de hoy."}
{"usuario":"Asistente Emocional","Pronombre":"Él","Mensaje":"Hola Jannette, que mal que el dia de hoy no te sientas bien, ¿Podrias contarme que es lo que te molesta?"}
{"usuario":"Jannette","Pronombre":"Ella","Mensaje":"En serio que como me cag

In [19]:
input=inputs[2]
prompt = f'''Below is an instruction that describes a task, paired with an input that provides further context. 

### Instruction:
{instructions[2]}\nskip emojis.

### Input:
{input}

### Response:'''
a=response(prompt,100)
print(a)

Numero de tokens de entrada:409

Salida:
Below is an instruction that describes a task, paired with an input that provides further context. 

### Instruction:
You are an emotional assistant, respond empathetically in Spanish to each of the messages. If the user seems sad or upset, you should offer words of encouragement. If the user seems happy or excited, the assistant should share that joy and respond enthusiastically. In all cases, the assistant should maintain a respectful tone, if possible, encouraging you to talk more about it. Don't say hello, unless necessary. Use the username and pronoun to respond in a personalized way.
skip emojis.

### Input:
{"usuario":"Jannette","Pronombre":"Ella","Mensaje":"Hola, me siento molesta el dio de hoy."}
{"usuario":"Asistente Emocional","Pronombre":"Él","Mensaje":"Hola Jannette, que mal que el dia de hoy no te sientas bien, ¿Podrias contarme que es lo que te molesta?"}
{"usuario":"Jannette","Pronombre":"Ella","Mensaje":"En serio que como me cag

In [ ]:
final.push_to_hub("Just_HEAR_Me")